In [ ]:
import pandas as pd

In [ ]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../Configuration"

In [ ]:
import refinitiv.data as rd

In [ ]:
rd.open_session()

In [ ]:
df_companies = pd.read_csv('Refinitiv_ESG_Universe.csv', delimiter=';')

df_companies.head()

In [ ]:
df_companies.info()

In [ ]:
# example, also works with ISIN instead of BASF Ticker
df = rd.get_data(
    universe=["BASFn.DE", "IBM.N"],
    fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
)

df

In [ ]:
# Stuff that doesn't change with time (Sector/Industry/Country/Region)
# testing the loop
test_frame = df_companies.iloc[:15, :]

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
comp_data_dict = {}

for isin in test_frame["ISIN"]:
    df = rd.get_data(
        universe=isin,
        fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
    )

    comp_data_dict[isin] = df

comp_dfs = []
for key, df in comp_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    comp_dfs.append(df)

# Concatenate the dataframes into a single one
comp_info_df_test = pd.concat(comp_dfs)

# save the new dataframe
comp_info_df_test.to_csv("comp_info_df_test.csv")

comp_info_df_test.head()

In [ ]:
# Stuff that doesn't change with time (Sector/Industry/Country/Region)

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
comp_data_dict = {}

for isin in df_companies["ISIN"]:
    
    df = rd.get_data(
        universe=isin,
        fields=["TR.HeadquartersCountry", "TR.HeadquartersRegion", "TR.NAICSNationalIndustry", "TR.NAICSIndustryGroup", 
            "TR.NAICSSector", "TR.GICSSubIndustry", "TR.GICSIndustry", "TR.ICBIndustry", "TR.ICBSector", "TR.GICSIndustryGroup", "TR.SICIndustryGroup"
            "TR.GICSSector", "TR.SICIndustry", "TR.TRBCIndustry", "TR.TRBCIndustryGroup", "TR.TRBCBusinessSector", "TR.TRBCEconomicSector",
            "TR.TSE33SectorNameMain", "TR.NAICSSubsector"]
    )

    comp_data_dict[isin] = df

comp_dfs = []
for key, df in comp_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    comp_dfs.append(df)

# Concatenate the dataframes into a single one
comp_info_df = pd.concat(comp_dfs)

# save the new dataframe
comp_info_df.to_csv("comp_info_df.csv")

In [ ]:
# testing the loop
test_frame = df_companies.iloc[:15, :]

# loop to collect data for all available companies, stored in a dictionary with a df for each isin
financials_data_dict = {}

for isin in test_frame["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
    )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    financials_data_dict[isin] = df

# Assume `dfs_dict` is the dictionary of dataframes
dfs = []
for key, df in financials_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    dfs.append(df)

# Concatenate the dataframes into a single one
result = pd.concat(dfs)

result.to_csv('comp_financials_test.csv')

result.head()

In [ ]:
# loop to collect data for all available companies, stored in a dictionary with a df for each isin
financials_data_dict = {}

for isin in df_companies["ISIN"]:

    df = rd.get_history(
        universe=isin,
        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
        interval="1Y",
        start="2000-01-01",
        end="2023-01-01",
    )
    
    df.index = pd.to_datetime(df.index)
    df = df.groupby(df.index.year).first()

    financials_data_dict[isin] = df

# Assume `dfs_dict` is the dictionary of dataframes
dfs = []
for key, df in financials_data_dict.items():
    df['key'] = key  # Add a new column containing the key
    dfs.append(df)

# Concatenate the dataframes into a single one
result = pd.concat(dfs)

result.to_csv('comp_financials.csv')

result.head()

In [ ]:
# Stuff that changes with time (Revenue/Earnings/Employees/ROA/ROE/Free Cash Flow)
# importing the dataframe with the relevant years
#ESG_TR_df = pd.read_csv("tr_esg_df_cleaned.csv")
#
#test_frame = ESG_TR_df.iloc[:30, :]
#
## loop to collect data for all available companies, stored in a dictionary with a df for each isin
#comp_financials_data_dict = {}
#
#for year, isin in zip(test_frame["Year"], test_frame["key"]):
#
#    df = rd.get_history(
#        universe=isin,
#        fields=["TR.RevenueMean", "TR.EarningsMean", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
#        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
#        interval="1Y",
#        start=str(year) + "-01-01",
#        end=str(year + 1) + "-01-01",
#    )
#    
#    df.index = pd.to_datetime(df.index)
#    df = df.groupby(df.index.year).first()
#
#    dict_key = isin + str(year)
#    comp_financials_data_dict[dict_key] = df
#
## Assume `dfs_dict` is the dictionary of dataframes
#dfs = []
#for key, df in comp_financials_data_dict.items():
#    df['key'] = key  # Add a new column containing the key
#    dfs.append(df)
#
## Concatenate the dataframes into a single one
#comp_financials_test = pd.concat(dfs)
#
## save the new dataframe
#comp_financials_test.to_csv('comp_financials_test.csv')
#
#comp_financials_test.head()

In [ ]:
# Stuff that changes with time (Revenue/Earnings/Employees/ROA/ROE/Free Cash Flow)
# importing the dataframe with the relevant years
#ESG_TR_df = pd.read_csv("tr_esg_df_cleaned.csv")
#
#test_frame = ESG_TR_df.iloc[:30, :]
#
## loop to collect data for all available companies, stored in a dictionary with a df for each isin
#comp_financials_data_dict = {}
#
#def financials_collection(year, isin):
#
#
#
#
#    df = rd.get_history(
#        universe=isin,
#        fields=["TR.Revenue", "TR.Earnings", "TR.F.EmpAvg", "TR.EPSMean", "TR.ROAMean", "TR.ROEMean",
#        "TR.Volume", "TR.CompanyMarketCap", "TR.FCFMean", "TR.WACC", "TR.EBITDAMean", "TR.OrganicSalesGrowthMean"],
#        interval="1Y",
#        start=str(year) + "-01-01",
#        end=str(year + 1) + "-01-01",
#    )
#    
#    df.index = pd.to_datetime(df.index)
#    df = df.groupby(df.index.year).first()
#
#    dict_key = isin + str(year)
#
#    comp_financials_data_dict[isin] = df
#
## Assume `dfs_dict` is the dictionary of dataframes
#dfs = []
#for key, df in comp_financials_data_dict.items():
#    df['key'] = key  # Add a new column containing the key
#    dfs.append(df)
#
## Concatenate the dataframes into a single one
#comp_financials_test = pd.concat(dfs)
#
## save the new dataframe
#comp_financials_test.to_csv('comp_financials_test.csv')
#
#comp_financials_test.head()

In [ ]:
rd.close_session()